## Projekt - Jakub Grzelak

## Transformer Encoder - RoBERTa

In [ ]:
!pip install 'transformers[torch]>=4.45,<4.46'
!pip install torch
!pip install datasets

In [ ]:
import transformers
import torch
from transformers import RobertaConfig, RobertaModel
from datasets import load_dataset

  Zbiór danych


In [ ]:
dataset = load_dataset("rotten_tomatoes")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})


Tokenizacja danych

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Model



RobertaForSequenceClassification

In [ ]:
from transformers import RobertaForSequenceClassification
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base')
print(roberta_model)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

Trenowanie

In [ ]:
import os
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score

os.environ["WANDB_DISABLED"] = "true"

train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": accuracy, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.382200,0.397108,0.833021,0.831478
2,0.204000,0.428483,0.872420,0.872290


TrainOutput(global_step=1068, training_loss=0.31981207765220254, metrics={'train_runtime': 1579.9307, 'train_samples_per_second': 10.798, 'train_steps_per_second': 0.676, 'total_flos': 4488674604441600.0, 'train_loss': 0.31981207765220254, 'epoch': 2.0})

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.42848312854766846, 'eval_accuracy': 0.8724202626641651, 'eval_f1': 0.8722903453136011, 'eval_runtime': 27.6786, 'eval_samples_per_second': 38.513, 'eval_steps_per_second': 0.614, 'epoch': 2.0}


Test na przykładzie

In [ ]:
new_texts = ["This movie was fantastic!", "I hated this film."]

inputs = tokenizer(new_texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

outputs = model(**inputs.to(model.device))
predictions = outputs.logits.argmax(dim=1)
print(predictions)  # 0 = negatywna, 1 = pozytywna


tensor([1, 0], device='cuda:0')


### Transformer Decoder - GPT-2


In [ ]:
import os
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
from datasets import load_dataset
import gc
gc.collect()

30

Tokenizacja danych

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

dataset = load_dataset("rotten_tomatoes")

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Model

GPT2ForSequenceClassification

In [ ]:
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]

model = GPT2ForSequenceClassification.from_pretrained(
    'gpt2',
    num_labels=2,
    pad_token_id=tokenizer.eos_token_id
)

print(model)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


Trenowanie

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.451000,0.373983,0.847092,0.847092
2,0.235100,0.519141,0.844278,0.844185
3,0.145000,0.675473,0.861163,0.861163
4,0.013200,0.928985,0.853659,0.853633


TrainOutput(global_step=4268, training_loss=0.28143949046684735, metrics={'train_runtime': 3567.4828, 'train_samples_per_second': 9.564, 'train_steps_per_second': 1.196, 'total_flos': 8915445130199040.0, 'train_loss': 0.28143949046684735, 'epoch': 4.0})

### Transformer Encoder-Decoder - T5

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")
print(model)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
!pip install datasets
from datasets import load_dataset
import os

dataset = load_dataset("rotten_tomatoes")

train_dataset = dataset["train"]
eval_dataset = dataset["test"]

In [ ]:
def generate_answer_with_model(model: T5ForConditionalGeneration, text: str, max_length: int = 10):
    encoded_input = tokenizer(text, return_tensors="pt")
    result = model.generate(**encoded_input, max_length=max_length)
    result_decode = tokenizer.decode(result[0], skip_special_tokens=True).lower()
    return result_decode

In [ ]:
import numpy as np
correct_answers = 0
total_examples = len(eval_dataset)

for example in eval_dataset:
    input_text = f"sst2 sentence: {example['text']}"
    model_answer = generate_answer_with_model(model, input_text)

    expected_answer = "positive" if example["label"] == 1 else "negative"
    correct = (model_answer == expected_answer)
    correct_answers += correct

    if np.random.rand() < 0.01:
        print(f"Text: {example['text']}")
        print(f"Model Answer: {model_answer}")
        print(f"Expected Answer: {expected_answer}")
        print(f"Correct: {correct}")
        print("-" * 50)

# Obliczenie accuracy
accuracy = correct_answers / total_examples * 100
print(f"Model Accuracy: {accuracy:.2f}% ({correct_answers}/{total_examples})")


Text: lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .
Model Answer: positive
Expected Answer: positive
Correct: True
--------------------------------------------------
Text: the wild thornberrys movie is pleasant enough and the message of our close ties with animals can certainly not be emphasized enough .
Model Answer: positive
Expected Answer: positive
Correct: True
--------------------------------------------------
Text: combines a comically dismal social realism with a farcically bawdy fantasy of redemption and regeneration .
Model Answer: positive
Expected Answer: positive
Correct: True
--------------------------------------------------
Text: bring on the sequel .
Model Answer: positive
Expected Answer: positive
Correct: True
--------------------------------------------------
Text: . . . one of the most entertaining monster movies in ages . . .
Model Answer: positive
Expected Answer: positive
Corr

### Transformer w trybie few-shot/zero-shot learning/prompting - FLAN-T5

In [ ]:
!pip install datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import numpy as np

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
print(model)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
dataset = load_dataset("rotten_tomatoes")
test_data = dataset["test"]

def query_model(example, model, tokenizer):
    input_text = f"Classify the sentiment of the following sentence as positive or negative:\nSentence: {example['text']}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    outputs = model.generate(**inputs, max_length=10)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).lower()

    expected = "positive" if example["label"] == 1 else "negative"
    correct = (answer == expected)
    return answer, expected, correct

# Odpytywanie modelu dla wszystkich przykładów w zbiorze testowym
correct_answers = 0
total_examples = len(test_data)

for example in test_data:
    answer, expected, correct = query_model(example, model, tokenizer)
    correct_answers += correct

    # Przykłady
    if np.random.rand() < 0.01:  # 1% zbioru
        print(f"Text: {example['text']}")
        print(f"Model Answer: {answer}")
        print(f"Expected Answer: {expected}")
        print(f"Correct: {correct}")
        print("-" * 50)

accuracy = correct_answers / total_examples * 100
print(f"Model Accuracy: {accuracy:.2f}% ({correct_answers}/{total_examples})")



README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Text: its use of the thriller form to examine the labyrinthine ways in which people's lives cross and change , buffeted by events seemingly out of their control , is intriguing , provocative stuff .
Model Answer: positive
Expected Answer: positive
Correct: True
--------------------------------------------------
Text: the film was immensely enjoyable thanks to great performances by both steve buscemi and rosario dawson . . .
Model Answer: positive
Expected Answer: positive
Correct: True
--------------------------------------------------
Text: manages to accomplish what few sequels can -- it equals the original and in some ways even betters it .
Model Answer: positive
Expected Answer: positive
Correct: True
--------------------------------------------------
Text: a deliciously nonsensical comedy about a city coming apart at its seams .
Model Answer: positive
Expected Answer: positive
Correct: True
--------------------------------------------------
Text: one of the best of a growing strai